# Assigment 4

## Xujin He, xh1131

In [1]:
import math
import numpy as np
import scipy.sparse as sp

import igl
import meshplot as mp

from math import sqrt
from IPython.display import IFrame

In [2]:
v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)

c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

In [3]:
def align_field(V, F, TT, soft_id, soft_value, llambda):
    assert(soft_id[0] > 0)
    assert(soft_id.shape[0] == soft_value.shape[0])

    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]

    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None]
        
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
  
    # Arrays for the entries of the matrix
    data = []
    ii = []
    jj = []
    
    index = 0
    for f in range(F.shape[0]):
        for ei in range(3): # Loop over the edges
            
            # Look up the opposite face
            g = TT[f, ei]
            
            # If it is a boundary edge, it does not contribute to the energy
            # or avoid to count every edge twice
            if g == -1 or f > g:
                continue
                
            # Compute the complex representation of the common edge
            e  = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            
            # Add the term conj(f)*ui - conj(g)*uj to the energy matrix
            data.append(ef);  ii.append(index); jj.append(f)
            data.append(-eg); ii.append(index); jj.append(g)

            index += 1
            
    
    sqrtl = sqrt(llambda)
    
    # Convert the constraints into the complex polynomial coefficients and add them as soft constraints
    
    # Rhs of the system
    b = np.zeros(index + soft_id.shape[0], dtype=complex)
    
    for ci in range(soft_id.shape[0]):
        f = soft_id[ci]
        v = soft_value[ci, :]
        
        # Project on the local frame
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        
        data.append(sqrtl); ii.append(index); jj.append(f)
        b[index] = c * sqrtl
        
        index += 1
    
    assert(b.shape[0] == index)
    
    
    # Solve the linear system
    A = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("csr")
    u = sp.linalg.spsolve(A.H @ A, A.H @ b)

    R = T1 * u.real[:,None] + T2 * u.imag[:,None]

    return R

In [4]:
def plot_mesh_field(V, F, R, constrain_faces):
    # Highlight in red the constrained faces
    col = np.ones_like(f)
    col[constrain_faces, 1:] = 0
    
    # Scaling of the representative vectors
    avg = igl.avg_edge_length(V, F)/2

    #Plot from face barycenters
    B = igl.barycenter(V, F)

    p = mp.plot(V, F, c=col)
    p.add_lines(B, B + R * avg)
    
    return p

# Q1. Tangent vector fields for scalar field design

In [5]:
def delete_columns_csr(matrix, cols_to_del):
    keep_index = np.logical_not(np.in1d(np.arange(matrix.shape[1]), cols_to_del))
    keep_index = np.where(keep_index)[0]
    return matrix[:, keep_index]
def delete_rows_csr(matrix, rows_to_del):
    keep_index = np.logical_not(np.in1d(np.arange(matrix.shape[0]), rows_to_del))
    keep_index = np.where(keep_index)[0]
    return matrix[keep_index, :]

def align_field_hard(V, F, TT, hard_id, hard_value):
    assert(hard_id[0] > 0)
    assert(hard_id.shape[0] == hard_value.shape[0])
    # compute perface local orth basis and normals, this is equivalent to the code:
    '''
    # Edges
    e1 = V[F[:, 1], :] - V[F[:, 0], :]
    e2 = V[F[:, 2], :] - V[F[:, 0], :]
    # Compute the local reference systems for each face, T1, T2
    T1 = e1 / np.linalg.norm(e1, axis=1)[:,None] 
    T2 =  np.cross(T1, np.cross(T1, e2))
    T2 /= np.linalg.norm(T2, axis=1)[:,None]
    '''
    T1, T2, _ = igl.local_basis(V, F)
    # Arrays for the entries of the matrix Q
    data = []
    ii = []
    jj = []
    index = 0

    bdata = []
    hard_id_list = hard_id.tolist()
    # Add per-edge constraint
    for f in range(F.shape[0]):
        for ei in range(3):
            # Look up the opposite face
            g = TT[f, ei]
            # If it is a boundary edge, it does not contribute to the energy; or avoid counting every edge twice
            if g == -1 or f > g:
                continue 
            # Compute the shared edge
            e = V[F[f, (ei+1)%3], :] - V[F[f, ei], :]
            # ef, eg are the shared edge vector expressed in each local basis of f and g
            vef = np.array([np.dot(e, T1[f, :]), np.dot(e, T2[f, :])])
            vef /= np.linalg.norm(vef)
            ef = (vef[0] + vef[1]*1j).conjugate()
            veg = np.array([np.dot(e, T1[g, :]), np.dot(e, T2[g, :])])
            veg /= np.linalg.norm(veg)
            eg = (veg[0] + veg[1]*1j).conjugate()
            # System Assembly - smoothness
            # both are free variables - add the constraint as ususal
            if (f not in hard_id_list and g not in hard_id_list):
                data.append(ef);  ii.append(index); jj.append(f)
                data.append(-eg); ii.append(index); jj.append(g)
                bdata.append(0); 
                index += 1
            # one of them is a free variable, then preserve the free variable in the system, subtract the fixed constraint from the rhs
            elif (f not in hard_id_list) and (g in hard_id_list):
                data.append(ef);  ii.append(index); jj.append(f)
                v = hard_value[hard_id_list.index(g), :]
                c = np.dot(v, T1[g, :]) + np.dot(v, T2[g, :])*1j
                bdata.append(c*eg);
                index += 1
            elif (f in hard_id_list and g not in hard_id_list):
                data.append(-eg); ii.append(index); jj.append(g)
                v = hard_value[hard_id_list.index(f), :]
                c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
                bdata.append(-c*ef);
                index += 1
            # both are fixed variables - just skip it
                
    Q = sp.coo_matrix((data, (ii, jj)), shape=(index, F.shape[0])).asformat("lil")
    b = np.array(bdata)
    Q = delete_columns_csr(Q, hard_id) # need to remove zero columns from Q - where the fixed 
    Q = Q.asformat("csr")
    # solve the linear system to get uf and ug
    u = sp.linalg.spsolve(Q.H @ Q, Q.H @ b)
    # now insert back the removed hard constraints
    insert_c = []
    hard_id_sorted = np.sort(hard_id);
    hard_value_sorted = hard_value[hard_id.argsort()]
    for ci in range(hard_id_sorted.shape[0]):
        f = hard_id_sorted[ci]
        v = hard_value_sorted[ci, :]
        c = np.dot(v, T1[f, :]) + np.dot(v, T2[f, :])*1j
        insert_c.append(c)
    u = np.insert(u, hard_id_sorted-np.arange(hard_id_sorted.shape[0]), insert_c) 
    # field vector uf is expressed as a complex number composed from xf, yf, ug ... from xg, yg
    # so we map from complex field back to R^2 
    R = T1 * u.real[:,None] + T2 * u.imag[:,None]
    return R

In [6]:
R = align_field_hard(v, f, tt, cf, c)
col = np.ones_like(f)
col[cf, 1:] = 0
B = igl.barycenter(v, f)
avg = igl.avg_edge_length(v, f)/2
p1 = mp.plot(v, f, c=col)
p1.add_lines(B, B+R*avg, shading={'line_color':'magenta'})
Bc = igl.barycenter(v, f[cf])
p1.add_lines(Bc, Bc+c*avg, shading={'line_color':'blue'})

p1.save("q1.html")
IFrame(src='q1.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Plot saved to file q1.html.


## Interactive Part 

**This part is not working on HTML since ipywidgets does not support Iframe, use jupyter notebook instead**

As the mesh plot above indicates, the results from the Hard Constraint (magenta) are very close the the Soft Constraint result (black), but obvious difference can be observed around the constraint triangles. In order to make obvious comparison, we can pick a small (like $\lambda=1$) that reduces the penalty factor for soft constraints. 

As we gradually increase $\lambda$ to infinity, the Soft Constraint field (black) seems converging to Hard Constraint field (magenta) and match the constraint values (cyan) at the constraint faces, which is the expected behavior - when we increase penalty, the Soft Constraint becomes "stricter".


In [7]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plot_interactive_field(V,F,TT,cons_id,cons_value,lpower):
    print("lambda =", 10**lpower)
    R = align_field(V,F,TT,cons_id,cons_value,10**lpower)
    p1 = plot_mesh_field(V, F, R, cons_id)

    # we now interpolate with HARD constraint instead, which will only give 
    R2 = align_field_hard(V, F, TT, cons_id, cons_value)
    B = igl.barycenter(V, F)
    avg = igl.avg_edge_length(V, F)/2
    p1.add_lines(B, B+R2*avg, shading={'line_color':'magenta'})

    Bc = igl.barycenter(V, F[cons_id])
    p1.add_lines(Bc, Bc+c*avg, shading={'line_color':'cyan'})

    # print out the hard constraint field
    print(np.vstack([R2, c]))

interact(plot_interactive_field, V=fixed(v), F=fixed(f), TT=fixed(tt), cons_id=fixed(cf), cons_value=fixed(c), lpower=(0,6,1))

interactive(children=(IntSlider(value=3, description='lpower', max=6), Output()), _dom_classes=('widget-intera…

<function __main__.plot_interactive_field(V, F, TT, cons_id, cons_value, lpower)>

# Q2: Reconstructing a scalar field from a vector field

$$
g = \left[\begin{array}{c}
f_x|_1 \\
\vdots \\
f_x|_{F} \\
f_y|_1 \\
\vdots \\
f_y|_{F} \\
f_z|_1 \\
\vdots \\
f_z|_{F} \\
\end{array}\right] =
\left[\begin{array}{c}
- G_1 -  \\
\vdots \\
- G_F -  \\
- G_{F+1} -  \\
\vdots \\
- G_{2F} -  \\
- G_{2F+1} -  \\
- \vdots \\
- G_{3F} -  \\
\end{array}
\right]
\left[\begin{array}{c}
s_1 \\
\vdots \\
s_V
\end{array}
\right] = Gs
$$

$$
\begin{aligned}
g_t =
\left[\begin{array}{c}
f_x|_t \\
f_y|_t \\
f_z|_t \\
\end{array}
\right]
=
\left[\begin{array}{c}
- G_t -  \\
- G_{F+t} -  \\
- G_{2F+t} -  \\
\end{array}
\right]
\left[\begin{array}{c}
s_1 \\
\vdots \\
s_V
\end{array}
\right]
= M_t s
\end{aligned}
$$

$$
\begin{aligned}
u_t = \left[\begin{array}{c}
u_{tx}\\ 
u_{ty}\\ 
u_{tz}
\end{array}\right]
\end{aligned}
$$

## Dealing with the least squares:

$$
\begin{aligned}
&\min_g \sum_{\text{face }t} A_t \|g_t - u_t\|^2 \\
=& \min_g \sum_{\text{face }t} A_t (g_t - u_t)^T(g_t - u_t) \\
=& \min_g \sum_{\text{face }t} A_t (g_t^Tg_t - u_t^Tg_t - g_t^Tu_t + u_t^Tu_t) \\
=& \min_s \sum_{\text{face }t} A_t (s^TG_t^TG_ts - 2G_t^Tu_ts + u_t^Tu_t)
\end{aligned}
$$

Sent gradient wrt $s$ to 0:

$$
\begin{aligned}
2A_tG_t^TG_ts - 2A_tG_t^Tu_t &= 0 \\
A_tG_t^TG_ts = A_tG_t^Tu_t
\end{aligned}
$$

<!-- \left[\begin{array}{c}
- G_t^T G_t -  \\
- G_{F+t}^T G_{F+t} -  \\
- G_{2F+t}^T G_{2F+t} -  \\
\end{array}
\right] -->
<!-- $$
\begin{aligned}
A_t
G_t^TG_t
\left[\begin{array}{c}
s_1 \\
\vdots \\
s_V
\end{array}
\right]
= 
A_t\left[\begin{array}{c}
| & | & | \\
G_t & G_{F+t} & G_{2F+t} \\
| & | & |
\end{array}
\right]
\left[\begin{array}{c}
u_{tx}\\ 
u_{ty}\\ 
u_{tz}
\end{array}\right]
\end{aligned}
$$ -->


Since all rows of $G$ are decoupled, we can combine $G_t$'s to get $K = G^TAG, b=G^TAu$

$$
\begin{aligned}
G^TAGs = G^TAu
\end{aligned}
$$

Where $A$ is diagonal matrix of triangles area, repeated 3 times to match $G_t, G_{F+t}, G_{2F+t}$

<!-- $$
\left[\begin{array}{c}
- G_1^T A_1 G_1 -  \\
\vdots \\
- G_F^T A_F G_F -  \\
- G_{F+1}^T A_1 G_{F+1} -  \\
\vdots \\
- G_{2F}^T A_F G_{2F} -  \\
- G_{2F+1}^T A_1 G_{2F+1} -  \\
\vdots \\
- G_{3F}^T A_F G_{3F} -  \\
\end{array}
\right]
\left[\begin{array}{c}
s_1 \\
\vdots \\
s_V
\end{array}
\right]
=
\text{diag}\left(
\begin{array}{c}
A_1 \\
\vdots \\
A_F \\
A_1 \\
\vdots \\
A_F \\
A_1 \\
\vdots \\
A_F \\
\end{array}
\right)
\left[\begin{array}{c}
| & & | & | & & | & | & & |\\
G_1 & \cdots & G_F & G_{F+1} & \cdots & G_{2F} & G_{2F+1} & \cdots & G_{3F}\\
| & & | & | & & | & | & & |
\end{array}
\right]
\left[\begin{array}{c}
u_{1x} \\
\vdots \\
u_{Fx} \\
u_{1y} \\
\vdots \\
u_{Fy} \\
u_{1z} \\
\vdots \\
u_{Fz} \\
\end{array}
\right]
$$ -->

## Poisson reconstruction error

The guiding vector field is plotted in red lines, and the computed gradient field is plotted in black lines

The poisson reconstruction error is computed per-face as:

$$\varepsilon_t = \|g_t - u_t\|_2 = \|Gs - u_t\|_2$$

And this is plotted as a color map on vertices.

### Note on Poisson reconstruction error

Another valid Poisson reconstruction error can be defined as a single value:

$$
\|G^TAs - Au\| 
$$

This is basically norm of the flattened gradient field - I computed this value as well

In [8]:
import numpy.matlib

# stack the flattened gradient vactor (3#F x 1) to a perface gradient (#F x 3)
def stack_gradient(G,s):
    gs = G@s # 3#F x 1
    Fn = G.shape[0] // 3;
    gt = np.zeros((Fn,3))
    for i in range(Fn):
        gt[i,0] = gs[i]
        gt[i,1] = gs[i + Fn]
        gt[i,2] = gs[i + 2*Fn]
    return gt
    
def reconstruct_scalar(V,F,u,print_poisson_err=False):
    # gradient of phi of each triag t (3#F x #V), sparse
    G = igl.grad(V, F)
    # extend vector of 2*Area (#F x 1) to sparse matrix, repeated 3 times
    d_area = igl.doublearea(V, F)
    A = sp.diags(np.matlib.repmat(d_area/2, 1, 3)[0])
    # flatten u into [u_x; u_y; u_z] three parts
    uu = np.hstack((u[:,0], u[:,1], u[:,2]))
    
    K = G.T @ A @ G # (#V x 3#F) x (3#V x 3#F) x (3#F x #V) = (#V x #V)
    b = G.T @ A @ uu # (#V x 3#F) x (3#F x 3#F) x (3#F x 1)

    # setting one side dirichlet bc: s_0 = 0
    Kd = K[1:,1:]
    bd = b[1:]
    
    # check singularity of K
    # print(numpy.linalg.cond(K.todense()))
    # print(numpy.linalg.cond(Kd.todense()))
    
    # solve linsys
    sd = sp.linalg.spsolve(Kd, bd)
    s = np.concatenate([np.array([0]),sd])

    # computed G@s
    perface_grad = stack_gradient(G,s)
    # compute perface poisson reconstruction error
    error = np.linalg.norm(perface_grad-u, axis=1)
    # the single value poisson reconstruction error
    if (print_poisson_err):
        print("Single-valued Poisson reconstruction error: ", np.linalg.norm(G@s - uu, 2))
    return s, error

- Visualization of computed scalar function and its gradient.

Red lines represent the gradient field; Black lines represent the original interpolated vector field. 

In [9]:
R = align_field_hard(v, f, tt, cf, c)
[s, e] = reconstruct_scalar(v,f,R,True)
# visualize the scalar function s by color map
p = mp.plot(v, f, c=s)
# compute and overlay gradient
G = igl.grad(v, f)
grad = stack_gradient(G, s)
avg = igl.avg_edge_length(v, f)/2
B = igl.barycenter(v, f)
p.add_lines(B, B + grad*avg, shading={"line_color":"red"})
p.add_lines(B, B + R*avg)
p.save("q2_1.html")
IFrame(src='q2_1.html', width=700, height=600)

Single-valued Poisson reconstruction error:  7.375992424336969


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Plot saved to file q2_1.html.


- Plots of the Poisson reconstruction error.

In [10]:
# visualize the poisson error by a color map
p2 = mp.plot(v, f, c=e)
p2.save("q2_2.html")
IFrame(src='q2_2.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Plot saved to file q2_2.html.


- An ASCII dump of the reconstructed scalar function 

In [11]:
# print out #V x 1 vector of the reconstructed scalar function
print(s)

[ 0.         -0.76446078  0.91827024  0.96196406  0.04628406 -0.03356961
  0.1166635   0.0375994   1.06933835  0.10824325  0.34406749  0.99538273
  0.60028518  0.37920076  0.26212548 -0.38586703 -0.30455482  0.09887735
  0.14013563 -0.06500472  0.47010329  0.31292944  0.58217911  0.18454032
  1.01814364  0.97105447  1.05592144  0.50693882  0.59984643  0.53276802
  0.31799488  0.73893937  0.57953849  0.19878317  0.1723607   0.00907722
  0.05661388 -0.12403086  0.14186179  0.21680932  0.09211278 -0.03750692
 -0.12641178 -0.58201624  0.14224591  0.12540661  0.1469902   0.195992
  0.08153612 -0.15066812 -0.20635038 -0.50386682 -0.19136987  0.00517524
  0.05415595  0.12011937  0.09515583  0.08696624  0.11197622 -0.07971778
 -0.007735   -0.01119985 -0.00718443  0.10580208 -0.31369651 -0.45287251
 -0.33441159  0.05587202  0.12412472  0.18782344  0.27934195  0.15011393
  0.20065495  0.28975587  0.25268915  0.41590342  0.54464477  0.43764354
  0.72893258  1.03978566  0.76909781  0.62105231  0.7

# 3. Harmonic and LSCM Parameterizations

- Visualization of the computed mapping functions and the gradients for harmonic mapping.


In [12]:
vh, fh = igl.read_triangle_mesh("data/camel_head.off")
tth, _ = igl.triangle_triangle_adjacency(fh)

def harmonic_parametrization(V, F, TT):
    # compute largest ordered boundary loops in the given mesh, 
    # F (#F x 3) => bnd (#b x 1)
    bnd = igl.boundary_loop(F)
    # V (#V x 3), bnd (#b x 1) => bnd_uv (#b x 2) 2D pos on the unit circle for bnd vertices
    bnd_uv = igl.map_vertices_to_circle(V, bnd)
    # solve Laplace equation with Dirichlet boundary conditions
    uv = igl.harmonic(V,F,bnd,bnd_uv,1)
    return uv

uv = harmonic_parametrization(vh, fh, tth)
# plot mesh with uv
us = uv[:,0]
vs = uv[:,1]
p = mp.subplot(vh, fh, c=us, uv=uv, s=[1, 2, 0])
mp.subplot(uv, fh, uv=uv, shading={"wireframe": True}, data=p, s=[1, 2, 1])
# compute and overlay gradient
Bh = igl.barycenter(vh, fh)
avgh = igl.avg_edge_length(vh, fh)/2
G = igl.grad(vh, fh)
Gu = stack_gradient(G, us)
Gv = stack_gradient(G, vs)
p2 = mp.plot(vh, fh, c=us)
p2.add_lines(Bh, Bh + Gu*avgh, shading={"line_color":"magenta"})
p2.add_lines(Bh, Bh + Gv*avgh, shading={"line_color":"yellow"})

p.save("q3_11.html")
IFrame(src='q3_11.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Plot saved to file q3_11.html.


In [13]:
p2.save("q3_12.html")
IFrame(src='q3_12.html', width=700, height=600)

Plot saved to file q3_12.html.


- Visualization of the computed mapping functions and the gradients for lscm mapping.

In [20]:
def lscm_parametrization(V, F, TT):
    bnd = igl.boundary_loop(F)
    # fix two points
    b = np.array([bnd[0], bnd[-1]])
    bc = np.array([[0., 0.],[1., 0.]])
    # Least-squares conformal map: bnd (#b x 1) boundary indices, bc (#b x 2) list of boundary values
    uv = igl.lscm(V,F,b,bc)
    uv = np.array(uv[1])
    return uv

uv = lscm_parametrization(vh, fh, tth)
us = uv[:,0]
vs = uv[:,1]
p = mp.subplot(vh, fh, uv=uv, c=us , s=[1, 2, 0])
mp.subplot(uv, fh, uv=uv, shading={"wireframe": True}, data=p, s=[1, 2, 1])
# compute and overlay gradient
Bh = igl.barycenter(vh, fh)
avgh = igl.avg_edge_length(vh, fh)/2
G = igl.grad(vh, fh)
Gu = stack_gradient(G, us)
Gv = stack_gradient(G, vs)
p2 = mp.plot(vh, fh, c=us)
p2.add_lines(Bh, Bh + Gu*avgh, shading={"line_color":"magenta"})
p2.add_lines(Bh, Bh + Gv*avgh, shading={"line_color":"yellow"})

p.save("q3_21.html")
IFrame(src='q3_21.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(1.9967555…

Plot saved to file q3_21.html.


In [15]:
p2.save("q3_22.html")
IFrame(src='q3_22.html', width=700, height=600)

Plot saved to file q3_22.html.


# 4. Editing a parameterization with vector fields

## Editing the parameterization

## Detecting problems with the parameterization

- Visualization of the edited parameterization.
- Visualization of flipped elements.
- An ASCII dump of the flipped triangle indices resulting from an edited harmonic parameterization of the mesh `irr4-cyl2.off` where the parameterization's `V` coordinate is replaced with a scalar field designed from the gradient vector constraints provided in `irr4-cyl2.constraints`

In [16]:
def edit_parameterization(v, f, tt, cf, c):
    # user-guided vector field and perface gradient
    R = align_field_hard(v, f, tt, cf, c)
    [s, _] = reconstruct_scalar(v, f, R)
    G = igl.grad(v, f)
    Gs = stack_gradient(G, s)
    # subsitute V by the gradient of the user-guided vector field
    uv = harmonic_parametrization(v, f, tt)
    uv = np.vstack((uv[:,0], s)).T
    return uv

v, f = igl.read_triangle_mesh("data/irr4-cyl2.off")
tt, _ = igl.triangle_triangle_adjacency(f)
c = np.loadtxt("data/irr4-cyl2.constraints")
cf = c[:, 0].astype(np.int64)
c = c[:, 1:]

uv = edit_parameterization(v, f, tt, cf, c)
# replace one of the U, V functions with a function obtained from a smooth user-guided vector field
# plot mesh with uv
avg = igl.avg_edge_length(v, f)/2
B = igl.barycenter(v, f)
G = igl.grad(v, f)
grad = stack_gradient(G, s)
p = mp.plot(v, f, uv=uv, shading={"wireframe": True})
p.add_lines(B, B + grad*avg, shading={"line_color": "cyan"})
p.save("q4_1.html")
IFrame(src='q4_1.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Plot saved to file q4_1.html.


In [17]:
p2 = mp.plot(uv, f, uv=uv, shading={"wireframe": True})
p2.save("q4_2.html")
IFrame(src='q4_2.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.15…

Plot saved to file q4_2.html.


- Visualization of flipped elements.

## Note on flipped element:

To determine which is the "correct" direction, we use the original representation and compute the normals to each triangle. 

Using the same cross product order (same clockwise direction) to compute the normals from the uv representation.

We then compute the angle between the "correct normals" and the normals from the uv representation. If that angle is larger than 180 degrees, 
then it means the triangle is flipped.

Discussed with some fellow students, some of them use `igl.doublearea(uv, f)` and pick the triangle with negative signed area to be flipped - this should gives exactly the opposite result as mine - I believe mine is better, as we don't know in the sign results from `igl.doublearea(uv, f)`, whether the positive or negative direction matches the direction in original mesh.

In [18]:
def check_flipped(f, v, uv):
    flipped = []
    for i in range(f.shape[0]):
        # normal from the original representation which is the "correct direction"
        x1 = v[f[i, 0]]
        x2 = v[f[i, 1]]
        x3 = v[f[i, 2]]
        e1 = x2 - x1
        e2 = x3 - x1
        nx = np.cross(e1, e2)
        # normal from the uv plane
        uv1 = uv[f[i, 0]]
        uv2 = uv[f[i, 1]]
        uv3 = uv[f[i, 2]]
        uv_e1 = np.hstack((uv2 - uv1, 0))
        uv_e2 = np.hstack((uv3 - uv1, 0))
        nuv = np.cross(uv_e1, uv_e2)
        # compute the angle between the normals, if the angle between is larger than 180 deg then it is flipped
        angle = np.arccos(nx.dot(nuv)/(np.linalg.norm(nx)*np.linalg.norm(nuv)))
        if angle > np.pi/2:
            flipped.append(i)
    return flipped

flipped = check_flipped(f, v, uv)
col = np.ones_like(f)
col[flipped, 1:] = 0
p = mp.plot(v, f, c=col, shading={"wireframe": True})
p.save("q4_3.html")
IFrame(src='q4_3.html', width=700, height=600)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

Plot saved to file q4_3.html.


- An ASCII dump of the flipped triangle indices resulting from an edited harmonic parameterization of the mesh `irr4-cyl2.off` where the parameterization's `V` coordinate is replaced with a scalar field designed from the gradient vector constraints provided in `irr4-cyl2.constraints`

In [19]:
print(flipped)

[6, 24, 25, 29, 71, 87, 88, 89, 91, 92, 93, 94, 104, 105, 302, 304, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 323, 325, 333, 336, 521, 524, 525, 526, 527, 528, 529, 537, 538, 540, 547, 548, 565, 566, 567, 568, 637, 638, 640, 641, 642, 643, 644, 645, 646, 647, 648, 649, 650, 651, 652, 653, 654, 655, 656, 657, 658, 659, 660, 661, 662, 663, 664, 669, 670, 732, 740, 741, 742, 743, 744, 745]
